<a href="https://colab.research.google.com/github/sumaiya08/medicaldata/blob/master/EDApart2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [0]:
auth.authenticate_user()
print('Authenticated')


In [0]:
project_id='eicudata'


Total number of hospitals in the database -

In [0]:
%%bigquery --project $project_id totalhosp
Select COUNT (DISTINCT hospitalid)
from `physionet-data.eicu_crd.hospital`;

In [19]:
totalhosp

,f0_
0,208


Total number of ICU'S - 

In [0]:
%%bigquery --project $project_id totalicu
SELECT COUNT (DISTINCT wardid )
FROM `physionet-data.eicu_crd.patient`


In [21]:
totalicu

,f0_
0,335


Procedures that have ICD 9/10 codes available -

In [0]:
%%bigquery --project $project_id icdcode
select diagnosisstring,icd9code,count(*) as count
from `physionet-data.eicu_crd.diagnosis` 
where icd9code is not null
group by diagnosisstring, icd9code


In [25]:
icdcode

,diagnosisstring,icd9code,count
0,cardiovascular|shock / hypotension|hypovolemic...,"785.59, R57.1, R58",3249
1,cardiovascular|diseases of the aorta|aortic an...,"441.4, I71.4",727
2,cardiovascular|diseases of the aorta|aortic an...,"441.9, I71.9",81
3,pulmonary|disorders of the airways|asthma / br...,"493.90, J45",7593
4,oncology|hematologic malignancy|leukemia|chron...,"204.10, C91.10",394
...,...,...,...
3928,renal|disorder of acid base|metabolic acidosis...,"276.2, E87.2",5
3929,surgery|neurosurgical issues|seizures|from tumor,"345.90, R56.9",1
3930,pulmonary|respiratory failure|acute respirator...,"518.81, J96.00",1
3931,surgery|respiratory failure|ventilatory failur...,"518.81, J96.00",1


Top 10 diseases common amongst patients -

In [0]:
%%bigquery --project $project_id topdiseases 
select apacheadmissiondx, count(apacheadmissiondx) as count
from `physionet-data.eicu_crd.patient` 
group by apacheadmissiondx 
order by count(apacheadmissiondx) desc
limit 10;

In [37]:
topdiseases

,apacheadmissiondx,count
0,,22996
1,"Sepsis, pulmonary",8862
2,"Infarction, acute myocardial (MI)",7228
3,"CVA, cerebrovascular accident/stroke",6647
4,"CHF, congestive heart failure",6617
5,"Sepsis, renal/UTI (including bladder)",5273
6,"Rhythm disturbance (atrial, supraventricular)",4827
7,Diabetic ketoacidosis,4825
8,Cardiac arrest (with or without respiratory ar...,4580
9,"CABG alone, coronary artery bypass grafting",4543


Identifying patients with heart diseases using related drug name - 

In [0]:
%%bigquery --project $project_id heartpatients 
select gender, age, ethnicity,apacheadmissiondx,drugname
from `physionet-data.eicu_crd.admissiondrug` a
join `physionet-data.eicu_crd.patient` b ON (
a.patientunitstayid =b.patientunitstayid
)
where drugname like '%CLOPIDOGREL%' # can also look for drug WARFARIN

In [42]:
heartpatients

,gender,age,ethnicity,apacheadmissiondx,drugname
0,Male,74,,"Endarterectomy, carotid",CLOPIDOGREL ...
1,Male,68,Hispanic,"Bleeding, upper GI",CLOPIDOGREL ...
2,Female,74,,"Endarterectomy, carotid",CLOPIDOGREL ...
3,Male,69,Caucasian,"Hematoma subdural, surgery for",CLOPIDOGREL ...
4,Male,53,Caucasian,Hemorrhage (for gastrointestinal bleeding GI-s...,CLOPIDOGREL ...
...,...,...,...,...,...
1724,Female,85,African American,"Hematologic medical, other",CLOPIDOGREL ...
1725,Male,77,Caucasian,"Infarction, acute myocardial (MI)",CLOPIDOGREL ...
1726,Male,73,African American,Seizures (primary-no structural brain disease),CLOPIDOGREL ...
1727,Male,73,African American,"CVA, cerebrovascular accident/stroke",CLOPIDOGREL ...
